In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
from torchvision.models import ResNet18_Weights
import torchvision.transforms as transforms
from torchvision.datasets import EMNIST
from torch.utils.data import random_split
from sklearn.metrics import precision_score, f1_score
import torch.nn.functional as F

import torch
import torch.nn.functional as F

def softmax_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = -1,
    gamma: float = 2,
    reduction: str = "mean",
) -> torch.Tensor:
    """
    Multiclass Focal Loss with softmax.

    Args:
        inputs: A float tensor of shape (batch_size, num_classes). These are the logits (raw, unnormalized scores) for each class.
        targets: A long tensor of shape (batch_size). Each value is the class label (0 to num_classes-1).
        alpha: (optional) Weighting factor in range (0,1) to balance positive vs negative examples. Default = -1 (no weighting).
        gamma: Exponent of the modulating factor (1 - p_t) to balance easy vs hard examples.
        reduction: 'none' | 'mean' | 'sum'.
                 'none': No reduction will be applied to the output.
                 'mean': The output will be averaged.
                 'sum': The output will be summed.

    Returns:
        Loss tensor with the reduction option applied.
    """
    # Convert logits to probabilities using softmax
    log_probs = F.log_softmax(inputs, dim=1)

    # Gather the log_probs corresponding to the correct classes
    targets_one_hot = F.one_hot(targets, num_classes=inputs.size(1)).float()
    log_probs = (log_probs * targets_one_hot).sum(dim=1)

    probs = torch.exp(log_probs)  # Convert log_probs back to probabilities

    # Focal loss factor
    focal_factor = (1 - probs) ** gamma

    # Focal loss calculation
    loss = -focal_factor * log_probs

    if alpha >= 0:
        alpha_t = alpha * targets_one_hot + (1 - alpha) * (1 - targets_one_hot)
        loss = alpha_t.sum(dim=1) * loss

    # Apply reduction
    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

class ImageClassifier:
    def __init__(self, network, optimizer, criterion, l2_lambda=0.01):
        self.network = network
        self.optimizer = optimizer
        self.criterion = criterion
        self.l2_lambda = l2_lambda
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.network.to(self.device)

    def _regularize(self, network, l2_lambda):
        # Compute L2 regularization
        l2_reg = 0.0
        for param in network.parameters():
            l2_reg += torch.norm(param, 2)
        return l2_lambda * l2_reg

    def compute_loss(self, outputs, targets, l2_lambda=0.01, regularize = False):
        # Compute the cross-entropy loss
        ce_loss = self.criterion(outputs, targets)

        if regularize:
            # Compute regularization loss
            l2_reg = self._regularize(self.network, l2_lambda)

            return ce_loss + l2_reg

        return ce_loss

    def compute_metrics(self, preds, targets):
        """Helper function to compute accuracy, precision, and F1 score."""
        # Ensure preds are already in label form (if not already converted)
        if preds.dim() > 1:  # Check if preds need reduction
            preds = preds.argmax(dim=1)  # Get the predicted labels

        preds = preds.cpu().numpy()  # Convert predictions to NumPy
        targets = targets.cpu().numpy()  # Convert true labels to NumPy

        # Compute accuracy
        accuracy = (preds == targets).mean()

        # Compute precision and F1 score using scikit-learn
        precision = precision_score(targets, preds, average='weighted', zero_division=0)
        f1 = f1_score(targets, preds, average='weighted')

        return accuracy, precision, f1

    def train(self, train_loader, val_loader, n_epochs=10, patience=3):
        best_val_loss = float('inf')
        current_patience = 0

        for epoch in range(n_epochs):
            # Train
            self.network.train()
            train_loss = 0.0
            all_preds = []
            all_targets = []

            # Use tqdm for progress bar and set dynamic description
            train_bar = tqdm(enumerate(train_loader), total=len(train_loader), desc=f'Training Epoch {epoch + 1}')
            for batch_idx, (data, target) in train_bar:
                data, target = data.to(self.device), target.to(self.device)
                self.optimizer.zero_grad()

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with loss and accuracy
                current_accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                train_bar.set_postfix(loss=train_loss / (batch_idx + 1), accuracy=current_accuracy)

            # Calculate final metrics for training
            all_preds = torch.cat(all_preds)
            all_targets = torch.cat(all_targets)
            train_accuracy, train_precision, train_f1 = self.compute_metrics(all_preds, all_targets)

            # Validate
            self.network.eval()
            val_loss = 0.0
            val_preds = []
            val_targets = []

            # Use tqdm for validation progress bar
            val_bar = tqdm(val_loader, desc='Validating')
            with torch.no_grad():
                for data, target in val_bar:
                    data, target = data.to(self.device), target.to(self.device)

                    # Forward pass
                    outputs = self.network(data)

                    # Compute loss
                    loss = self.compute_loss(outputs, target)
                    val_loss += loss.item()

                    # Gather predictions and true labels
                    preds = outputs.argmax(dim=1)
                    val_preds.append(preds)
                    val_targets.append(target)

                    # Update progress bar with validation loss and accuracy
                    val_accuracy, _, _ = self.compute_metrics(torch.cat(val_preds), torch.cat(val_targets))
                    val_bar.set_postfix(val_loss=val_loss / len(val_loader), accuracy=val_accuracy)

            # Calculate final validation metrics
            val_preds = torch.cat(val_preds)
            val_targets = torch.cat(val_targets)
            val_accuracy, val_precision, val_f1 = self.compute_metrics(val_preds, val_targets)

            # Print epoch statistics
            train_loss /= len(train_loader)
            val_loss /= len(val_loader)
            print(f'Epoch {epoch + 1}/{n_epochs}, '
                  f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, '
                  f'Train Acc: {train_accuracy:.4f}, Val Acc: {val_accuracy:.4f}, '
                  f'Train Prec: {train_precision:.4f}, Val Prec: {val_precision:.4f}, '
                  f'Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')

            # Check for early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                current_patience = 0
            else:
                current_patience += 1
                if current_patience >= patience:
                    print(f'Validation loss did not improve for {patience} epochs. Stopping training.')
                    break

    def test(self, test_loader):
        self.network.eval()
        test_loss = 0.0
        correct = 0
        all_preds = []
        all_targets = []

        # Use tqdm for test progress bar
        test_bar = tqdm(test_loader, desc='Testing')
        with torch.no_grad():
            for data, target in test_bar:
                data, target = data.to(self.device), target.to(self.device)

                # Forward pass
                outputs = self.network(data)

                # Compute loss
                loss = self.compute_loss(outputs, target)
                test_loss += loss.item()

                # Gather predictions and true labels for accuracy/metrics calculation
                preds = outputs.argmax(dim=1)
                all_preds.append(preds)
                all_targets.append(target)

                # Update progress bar with test loss and accuracy
                accuracy, _, _ = self.compute_metrics(torch.cat(all_preds), torch.cat(all_targets))
                test_bar.set_postfix(loss=test_loss / len(test_loader), accuracy=accuracy)

        # Calculate final test metrics
        all_preds = torch.cat(all_preds)
        all_targets = torch.cat(all_targets)
        accuracy, precision, f1 = self.compute_metrics(all_preds, all_targets)

        test_loss /= len(test_loader)
        print(f'Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%, Precision: {precision:.2f}, F1 Score: {f1:.2f}')

# Define transformation for the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to 224x224
    transforms.ToTensor(),            # Convert to tensor (1 channel)
    transforms.Lambda(lambda x: x.repeat(3, 1, 1)),  # Convert 1 channel to 3 channels (RGB)
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize for RGB
])

# Download the EMNIST ByClass dataset
emnist_dataset = EMNIST(root='data', split='byclass', train=True, download=True, transform=transform)
test_dataset = EMNIST(root='data', split='byclass', train=False, download=True, transform=transform)

# Define the sizes for the training and validation sets
train_size = int(0.85 * len(emnist_dataset))  # 80% for training
val_size = len(emnist_dataset) - train_size   # remaining 20% for validation

# Split the dataset into training and validation sets
train_dataset, val_dataset = random_split(emnist_dataset, [train_size, val_size])

print(f'Training set size: {len(train_dataset)}')
print(f'Validation set size: {len(val_dataset)}')
print(f'Test set size: {len(test_dataset)}')

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=448, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=448)
test_loader = DataLoader(test_dataset, batch_size=448)

# Example neural network architecture using ResNet-18
class ResNet18Classifier(nn.Module):
    def __init__(self, num_classes=62):
        super(ResNet18Classifier, self).__init__()
        self.resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)

        for name, child in self.resnet.named_children():
            if name in ['layer1', 'layer2', 'layer3']:
                for param in child.parameters():
                    param.requires_grad = False

        self.resnet.fc = nn.Linear(self.resnet.fc.in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Initialize the neural network, optimizer, and criterion
model = ResNet18Classifier(num_classes=62)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = softmax_focal_loss

# Create an instance of ImageClassifier
classifier = ImageClassifier(model, optimizer, criterion)

# Train the classifier
classifier.train(train_loader, val_loader, n_epochs=10, patience=3)

# Test the classifier
classifier.test(test_loader)

100%|██████████| 561753746/561753746 [00:09<00:00, 58311998.54it/s]


Extracting data/EMNIST/raw/gzip.zip to data/EMNIST/raw
Training set size: 593242
Validation set size: 104690
Test set size: 116323


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 71.7MB/s]
Validating: 100%|██████████| 234/234 [04:07<00:00,  1.06s/it, accuracy=0.856, val_loss=0.173]


Epoch 1/10, Train Loss: 0.1971, Val Loss: 0.1726, Train Acc: 0.8506, Val Acc: 0.8559, Train Prec: 0.8387, Val Prec: 0.8560, Train F1: 0.8405, Val F1: 0.8441


Validating: 100%|██████████| 234/234 [04:05<00:00,  1.05s/it, accuracy=0.843, val_loss=0.189]


...

Epoch 3/10, Train Loss: 0.1362, Val Loss: 0.1574, Train Acc: 0.8749, Val Acc: 0.8654, Train Prec: 0.8668, Val Prec: 0.8647, Train F1: 0.8654, Val F1: 0.8527


Validating: 100%|██████████| 234/234 [04:09<00:00,  1.07s/it, accuracy=0.857, val_loss=0.163]


Epoch 4/10, Train Loss: 0.1272, Val Loss: 0.1631, Train Acc: 0.8793, Val Acc: 0.8567, Train Prec: 0.8719, Val Prec: 0.8609, Train F1: 0.8702, Val F1: 0.8431


Training Epoch 5:   0%|          | 3/1325 [00:05<39:14,  1.78s/it, accuracy=0.882, loss=0.136]


KeyboardInterrupt: 

In [ ]:
classifier.test(test_loader)

Testing: 100%|██████████| 260/260 [04:28<00:00,  1.03s/it, accuracy=0.839, loss=0.186]


Test Loss: 0.1865, Accuracy: 0.84%, Precision: 0.85, F1 Score: 0.82


In [ ]:
# Save the model after training
torch.save(classifier.network.state_dict(), 'resnet18_classifier_setting3.pth')